In [1]:
import numpy as np

In [2]:
A = np.array([
    [1, -1, 2, -1],
    [2, 0, 3, 1],
    [1, 1, 3, -1],
    [2, 1, 5, -2]
], dtype=np.float32)

b = np.array([1, 4, 2, 3])

# 1. Метод Гаусса

In [3]:
def triangle_view(matrix: np.ndarray, eps: float = 1e-7) -> np.ndarray:
    A = matrix.copy()
    N = A.shape[0]
    for i in range(N - 1):
        if abs(A[i, i]) < eps:
            for k in range(i + 1, N):
                if abs(A[k, i]) > eps:
                    A[i], A[k] = A[k], A[i]
                    break
        for j in range(i + 1, N):
            d = A[j, i] / A[i, i]
            A[j] -= d * A[i]
    return A


Формула вычисления корней: $$x_i = \frac{1}{a_{ii}}(b_i - \sum\limits_{j=0}^{n - i - 1}{a_{in-j} \cdot x_{n-j}})$$

In [4]:
def get_roots(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    n = b.shape[0]
    roots = np.zeros(n)
    roots[n - 1] += b[n - 1] / A[n - 1, n - 1]
    for i in range(n - 2, -1, -1):
        m = 0
        for j in range(n - i):
            m += A[i, n - j - 1] * roots[n - j - 1]
        roots[i] += (b[i] - m) / A[i, i]
    return roots

In [5]:
def gauss_method(A: np.ndarray, b: np.ndarray, eps: float = 1e-7) -> np.ndarray:
    forward_pass = triangle_view(np.c_[A, b], eps)
    return get_roots(forward_pass[:, :-1], forward_pass[:, -1])

gauss_method(A, b)

array([0., 0., 1., 1.])

# 2. Метод обратной матрицы

In [6]:
def inverse_matrix_method(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    return np.linalg.inv(A) @ b


np.around(inverse_matrix_method(A, b))

array([ 0., -0.,  1.,  1.])

# 3. Метод прогонки

In [7]:
matrix = np.array([
    [0, 3, 1, 5],
    [1, 2, 1, 6],
    [3, 9, 6, 25],
    [2, 4, 0, 5]
])

In [8]:
def sweep_method(matrix: np.ndarray) -> np.ndarray:
    n = matrix.shape[0]
    U = [-matrix[0, 2]/matrix[0, 1]]
    V = [matrix[0, 3]/matrix[0, 1]]
    for i in range(1, n):
        U.append(-matrix[i, 2]/(matrix[i, 0] * U[i - 1] + matrix[i, 1]))
        V.append((matrix[i, 3] - matrix[i, 0] * V[i - 1])/(matrix[i, 0] * U[i - 1] + matrix[i, 1]))
    X = np.zeros(n)
    X[-1] += V[-1]
    for i in range(n - 2, -1, -1):
        X[i] += U[i] * X[i + 1] + V[i]
    R = np.zeros(n)
    R[0] += matrix[0, 3] - matrix[0, 1] * X[0] - matrix[0, 2] * X[1]
    R[-1] += matrix[-1, 3] - matrix[-1, 0] * X[-2] - matrix[-1, 1] * X[-1]
    for i in range(1, n - 1):
        R[i] += matrix[i, 3] - matrix[i, 0] * X[i - 1] - matrix[i, 1] * X[i] - matrix[i, 2] * X[i + 1]
    return X, R

X, R = sweep_method(matrix)
print(f"1) Решение: {X}\n2) Невязка: {np.around(R)}")

1) Решение: [1.26190476 1.21428571 2.30952381 0.0952381 ]
2) Невязка: [-0.  0.  0.  0.]


# 4. Итерационные методы

In [9]:
M = np.array([
    [3, 1, -1, 1],
    [1, -4, 1, -1],
    [-1, 1, 4, 1],
    [1, 2, 1, -5]
])

d = np.array([6, -4, 13, 4])

In [30]:
def jacobi_method(A: np.ndarray, b: np.ndarray, x: np.ndarray, k_max: int = 100) -> np.ndarray:
    n = A.shape[1]
    x1 = x
    x2 = x
    k = 0
    X = None
    while True:
        if k == k_max:
            X = x2
            break

        for i in range(n):
            s = 0
            for j in range(n):
                s += A[i, j] * x1[j]
            x2[i] = x1[i] + (b[i] - s) / A[i, i]
        
        x1 = x2

        k += 1

    return X, b - M @ X


X, R = jacobi_method(M, d, np.array([0, 0, 0, 0]))
print(f"1) Решение: {X}\n2) Невязка: {np.around(R)}")

1) Решение: [2 2 3 1]
2) Невязка: [0 0 0 0]


In [31]:
def seidel_method(A: np.ndarray, b: np.ndarray, x: np.ndarray, k_max: int = 100) -> np.ndarray:
    n = A.shape[1]
    x1 = x
    k = 0
    X = None
    while True:
        if k == k_max:
            X = x1
            break

        for i in range(n):
            s = 0
            for j in range(n):
                s += A[i, j] * x1[j]
            x1[i] += (b[i] - s) / A[i, i]

        k += 1

    return X, b - M @ X


X, R = seidel_method(M, d, np.array([0, 0, 0, 0]))
print(f"1) Решение: {X}\n2) Невязка: {np.around(R)}")

1) Решение: [2 2 3 1]
2) Невязка: [0 0 0 0]


# 5. Нахождение собственных чисел матрицы

In [ ]:
def matrix_eigenvalues(A: np.ndarray) -> np.ndarray:
    ...